In [1]:
import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
spark = SparkSession.builder \
    .appName("loan_prediction") \
    .config("spark.jars", "/Users/ramazanyildirim/.vscode/extensions/oracle.sql-developer-25.1.0-darwin-x64/dbtools/sqlcl/lib/modules/ojdbc11-23.7.0.25.01.jar") \
    .getOrCreate()

25/04/28 15:03:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
spark

In [4]:
df = spark.read.format("jdbc") \
    .option("url", "jdbc:oracle:thin:@localhost:1521/xe") \
    .option("user", "system") \
    .option("password", "Oracle123") \
    .option("dbtable", "loanprediction") \
    .option("driver", "oracle.jdbc.OracleDriver") \
    .load()


In [5]:
df.show()

25/04/28 15:03:47 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


+--------+------+-------+----------+------------+-------------+---------------+-----------------+--------------+----------------+--------------+-------------+-----------+
| Loan_ID|Gender|Married|Dependents|   Education|Self_Employed|ApplicantIncome|CoapplicantIncome|    LoanAmount|Loan_Amount_Term|Credit_History|Property_Area|Loan_Status|
+--------+------+-------+----------+------------+-------------+---------------+-----------------+--------------+----------------+--------------+-------------+-----------+
|LP001002|  Male|     No|         0|    Graduate|           No|           5849|     0.0000000000|          NULL|  360.0000000000|  1.0000000000|        Urban|          Y|
|LP001003|  Male|    Yes|         1|    Graduate|           No|           4583|  1508.0000000000|128.0000000000|  360.0000000000|  1.0000000000|        Rural|          N|
|LP001005|  Male|    Yes|         0|    Graduate|          Yes|           3000|     0.0000000000| 66.0000000000|  360.0000000000|  1.0000000000| 

In [6]:
import pandas as pd
b = pd.read_csv("Loan Prediction Dataset.csv")
b

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,LP002978,Female,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y
610,LP002979,Male,Yes,3+,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y
611,LP002983,Male,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y
612,LP002984,Male,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y


In [7]:
df.printSchema()

root
 |-- Loan_ID: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Married: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- Education: string (nullable = true)
 |-- Self_Employed: string (nullable = true)
 |-- ApplicantIncome: decimal(38,0) (nullable = true)
 |-- CoapplicantIncome: decimal(38,10) (nullable = true)
 |-- LoanAmount: decimal(38,10) (nullable = true)
 |-- Loan_Amount_Term: decimal(38,10) (nullable = true)
 |-- Credit_History: decimal(38,10) (nullable = true)
 |-- Property_Area: string (nullable = true)
 |-- Loan_Status: string (nullable = true)



In [8]:
df.dtypes

[('Loan_ID', 'string'),
 ('Gender', 'string'),
 ('Married', 'string'),
 ('Dependents', 'string'),
 ('Education', 'string'),
 ('Self_Employed', 'string'),
 ('ApplicantIncome', 'decimal(38,0)'),
 ('CoapplicantIncome', 'decimal(38,10)'),
 ('LoanAmount', 'decimal(38,10)'),
 ('Loan_Amount_Term', 'decimal(38,10)'),
 ('Credit_History', 'decimal(38,10)'),
 ('Property_Area', 'string'),
 ('Loan_Status', 'string')]

In [9]:
df.show(5)

+--------+------+-------+----------+------------+-------------+---------------+-----------------+--------------+----------------+--------------+-------------+-----------+
| Loan_ID|Gender|Married|Dependents|   Education|Self_Employed|ApplicantIncome|CoapplicantIncome|    LoanAmount|Loan_Amount_Term|Credit_History|Property_Area|Loan_Status|
+--------+------+-------+----------+------------+-------------+---------------+-----------------+--------------+----------------+--------------+-------------+-----------+
|LP001002|  Male|     No|         0|    Graduate|           No|           5849|     0.0000000000|          NULL|  360.0000000000|  1.0000000000|        Urban|          Y|
|LP001003|  Male|    Yes|         1|    Graduate|           No|           4583|  1508.0000000000|128.0000000000|  360.0000000000|  1.0000000000|        Rural|          N|
|LP001005|  Male|    Yes|         0|    Graduate|          Yes|           3000|     0.0000000000| 66.0000000000|  360.0000000000|  1.0000000000| 

# Data Analysis

In [10]:
# Loan_Status sütununu aggregate edelim
df.groupBy("Loan_Status").count().show()

+-----------+-----+
|Loan_Status|count|
+-----------+-----+
|          Y|  422|
|          N|  192|
+-----------+-----+



In [11]:
# DataFrame üzerinde gruplama ve toplama işlemini gerçekleştirelim
df.select("Credit_History", "Loan_Status").groupBy("Loan_Status").agg(F.avg("Credit_History")).show()

#df.select( "Loan_Status","Credit_History",).groupBy('Loan_Status').agg(F.avg('Credit_History')).show() --> Aynı şeyi verecek çıktı

+-----------+-------------------+
|Loan_Status|avg(Credit_History)|
+-----------+-------------------+
|          Y|   0.98181818181818|
|          N|   0.54189944134078|
+-----------+-------------------+



Yukarıdaki kod parçacığı, DataFrame'den "Credit_History" ve "Loan_Status" sütunlarını seçer, verileri "Loan_Status" sütununa göre gruplandırır ve ardından her grup için "Credit_History" sütununun ortalamasını hesaplar

In [12]:
# DataFrame üzerinde gruplama ve sayma işlemini gerçekleştirelim
df.select("Gender", "Loan_Status").groupBy("Loan_Status", "Gender").count().show()

+-----------+------+-----+
|Loan_Status|Gender|count|
+-----------+------+-----+
|          N|Female|   37|
|          Y|  NULL|    8|
|          Y|Female|   75|
|          N|  NULL|    5|
|          Y|  Male|  339|
|          N|  Male|  150|
+-----------+------+-----+



Veri Çerçevesinden 'Cinsiyet' ve 'Kredi_Durumu' sütunlarını seçer, verileri hem 'Kredi_Durumu' hem de 'Cinsiyet'e göre gruplandırır ve ardından her kombinasyon için oluşum sayısını sayar

# Correlation Matrix

In [13]:
columns = ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term', 'Credit_History']
corr_df = pd.DataFrame()
for i in columns:
    corr = []
    for j in columns:
        corr.append(round(df.stat.corr(i, j), 2))
    corr_df = pd.concat([corr_df, pd.Series(corr)], axis=1)
corr_df.columns = columns
corr_df.insert(0, '', columns)
corr_df.set_index('')

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
,,,,,
ApplicantIncome,1.00,-0.12,0.54,-0.02,0.01
CoapplicantIncome,-0.12,1.00,0.19,-0.05,-0.06
LoanAmount,0.54,0.19,1.00,0.06,-0.03
Loan_Amount_Term,-0.02,-0.05,0.06,1.00,0.05
Credit_History,0.01,-0.06,-0.03,0.05,1.00


# SQL İşlemleri

In [14]:
import pyspark.sql as sparksql

Apache Spark'taki DataFrame df'den "table" adlı geçici bir view oluşturacağız. Geçici bir view, SQL benzeri sözdizimini kullanarak DataFrame'i sorgulamanıza olanak tanır


In [15]:
df.createOrReplaceTempView("table")

In [16]:
# Tablodaki ilk 5 satırı getirelim
spark.sql("select * from table limit 5").show()

+--------+------+-------+----------+------------+-------------+---------------+-----------------+--------------+----------------+--------------+-------------+-----------+
| Loan_ID|Gender|Married|Dependents|   Education|Self_Employed|ApplicantIncome|CoapplicantIncome|    LoanAmount|Loan_Amount_Term|Credit_History|Property_Area|Loan_Status|
+--------+------+-------+----------+------------+-------------+---------------+-----------------+--------------+----------------+--------------+-------------+-----------+
|LP001002|  Male|     No|         0|    Graduate|           No|           5849|     0.0000000000|          NULL|  360.0000000000|  1.0000000000|        Urban|          Y|
|LP001003|  Male|    Yes|         1|    Graduate|           No|           4583|  1508.0000000000|128.0000000000|  360.0000000000|  1.0000000000|        Rural|          N|
|LP001005|  Male|    Yes|         0|    Graduate|          Yes|           3000|     0.0000000000| 66.0000000000|  360.0000000000|  1.0000000000| 

In [17]:
spark.sql("select Loan_ID from table where Credit_History = 1").show()

+--------+
| Loan_ID|
+--------+
|LP001002|
|LP001003|
|LP001005|
|LP001006|
|LP001008|
|LP001011|
|LP001013|
|LP001018|
|LP001020|
|LP001024|
|LP001027|
|LP001028|
|LP001029|
|LP001030|
|LP001032|
|LP001038|
|LP001041|
|LP001046|
|LP001066|
|LP001068|
+--------+
only showing top 20 rows



# Data Preprocessing

In [18]:
#İlk olarak sütunlardaki boş değerleri görelim
df.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+-------+------+-------+----------+---------+-------------+---------------+-----------------+----------+----------------+--------------+-------------+-----------+
|Loan_ID|Gender|Married|Dependents|Education|Self_Employed|ApplicantIncome|CoapplicantIncome|LoanAmount|Loan_Amount_Term|Credit_History|Property_Area|Loan_Status|
+-------+------+-------+----------+---------+-------------+---------------+-----------------+----------+----------------+--------------+-------------+-----------+
|      0|    13|      3|        15|        0|           32|              0|                0|        22|              14|            50|            0|          0|
+-------+------+-------+----------+---------+-------------+---------------+-----------------+----------+----------------+--------------+-------------+-----------+



Bu, DataFrame df'deki her sütun için boş değerlerin sayısını saymak ve sonucu görüntülemek için Apache Spark'taki select işlevini kullanır


In [19]:
#sütunun ortalama değerini hesaplayacağız
mean = df.select(F.mean(df['LoanAmount'])).collect()[0][0]
mean = float(mean)
mean

146.41216216216216

pyspark.sql.functions modülündeki mean() fonksiyonunu kullanarak DataFrame df'deki 'LoanAmount' sütununun ortalama değerini hesaplar 146.41216216216216 Bu, 'LoanAmount' sütununun ortalama değeridir


In [20]:
# boş değerleri dolduracağız
df = df.na.fill(mean, ['LoanAmount'])

In [21]:
#bir sütunun mod değerini alacağız
df.groupby("Gender").count().orderBy("count", ascending = False).first()[0]

'Male'

DataFrame df'nin 'Cinsiyet' sütununda bir gruplama işlemi gerçekleştirir, her cinsiyet için oluşum sayısını sayar, sonucu sayıma göre azalan düzende sıralar ve ilk satırı alır. Son olarak, ilk satırın ilk sütununda en yüksek sayıma sahip cinsiyete karşılık gelen değeri döndürür

In [22]:
#Next we will fill the null values for all the columns
numerical_cols = ['LoanAmount', 'Loan_Amount_Term','Credit_History']
categorical_cols = ['Gender', 'Married', 'Dependents', 'Self_Employed']

In [23]:
for col in numerical_cols:
    mean = df.select(F.mean(df[col])).collect()[0][0]
    df = df.na.fill(float(mean), [col])

DataFrame df'nin birden fazla sayısal sütunundaki boş değerleri ilgili ortalama değerleriyle doldurur. Numerical_cols listesindeki her sütun adı üzerinde yineleme yapar, her sütun için ortalama değeri hesaplar ve ardından o sütundaki boş değerleri ilgili ortalama değerle doldurur

In [24]:
for col in categorical_cols:
    mode = df.groupby(col).count().orderBy("count", ascending=False).first()[0]
    df = df.na.fill(str(mode), [col])

DataFrame df'nin birden fazla kategorik sütunundaki boş değerleri ilgili mod değerleriyle doldurur. categorical_cols listesindeki her sütun adı üzerinde yineleme yapar, her sütun için mod değerini hesaplar ve ardından o sütundaki boş değerleri ilgili mod değeriyle doldurur

In [25]:
#Şimdi tekrar null değerlere bakalım
df.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+-------+------+-------+----------+---------+-------------+---------------+-----------------+----------+----------------+--------------+-------------+-----------+
|Loan_ID|Gender|Married|Dependents|Education|Self_Employed|ApplicantIncome|CoapplicantIncome|LoanAmount|Loan_Amount_Term|Credit_History|Property_Area|Loan_Status|
+-------+------+-------+----------+---------+-------------+---------------+-----------------+----------+----------------+--------------+-------------+-----------+
|      0|     0|      0|         0|        0|            0|              0|                0|         0|               0|             0|            0|          0|
+-------+------+-------+----------+---------+-------------+---------------+-----------------+----------+----------------+--------------+-------------+-----------+



In [26]:
# Yeni sütun ekleyelim
df = df.withColumn('TotalIncome', F.col('ApplicantIncome') + F.col('CoapplicantIncome'))
df.show(2)

+--------+------+-------+----------+---------+-------------+---------------+-----------------+--------------+----------------+--------------+-------------+-----------+-----------+
| Loan_ID|Gender|Married|Dependents|Education|Self_Employed|ApplicantIncome|CoapplicantIncome|    LoanAmount|Loan_Amount_Term|Credit_History|Property_Area|Loan_Status|TotalIncome|
+--------+------+-------+----------+---------+-------------+---------------+-----------------+--------------+----------------+--------------+-------------+-----------+-----------+
|LP001002|  Male|     No|         0| Graduate|           No|           5849|     0.0000000000|146.4121621622|  360.0000000000|  1.0000000000|        Urban|          Y|5849.000000|
|LP001003|  Male|    Yes|         1| Graduate|           No|           4583|  1508.0000000000|128.0000000000|  360.0000000000|  1.0000000000|        Rural|          N|6091.000000|
+--------+------+-------+----------+---------+-------------+---------------+-----------------+------

withColumn() fonksiyonu, DataFrame df'ye 'TotalIncome' adlı yeni bir sütun eklemek için kullanılır. Yeni sütun, F.col('ApplicantIncome') + F.col('CoapplicantIncome') toplama işlemini gerçekleştirerek oluşturulur; burada F.col(), karşılık gelen sütunlara başvurmak için kullanılır

In [27]:
# değerleri sayısallaştıralım
df = df.withColumn('Loan_Status', F.when(df['Loan_Status']=='Y', 1).otherwise(0))
df.show(2)

+--------+------+-------+----------+---------+-------------+---------------+-----------------+--------------+----------------+--------------+-------------+-----------+-----------+
| Loan_ID|Gender|Married|Dependents|Education|Self_Employed|ApplicantIncome|CoapplicantIncome|    LoanAmount|Loan_Amount_Term|Credit_History|Property_Area|Loan_Status|TotalIncome|
+--------+------+-------+----------+---------+-------------+---------------+-----------------+--------------+----------------+--------------+-------------+-----------+-----------+
|LP001002|  Male|     No|         0| Graduate|           No|           5849|     0.0000000000|146.4121621622|  360.0000000000|  1.0000000000|        Urban|          1|5849.000000|
|LP001003|  Male|    Yes|         1| Graduate|           No|           4583|  1508.0000000000|128.0000000000|  360.0000000000|  1.0000000000|        Rural|          0|6091.000000|
+--------+------+-------+----------+---------+-------------+---------------+-----------------+------

# Feature Engineering

In [28]:
df.printSchema()

root
 |-- Loan_ID: string (nullable = true)
 |-- Gender: string (nullable = false)
 |-- Married: string (nullable = false)
 |-- Dependents: string (nullable = false)
 |-- Education: string (nullable = true)
 |-- Self_Employed: string (nullable = false)
 |-- ApplicantIncome: decimal(38,0) (nullable = true)
 |-- CoapplicantIncome: decimal(38,10) (nullable = true)
 |-- LoanAmount: decimal(38,10) (nullable = true)
 |-- Loan_Amount_Term: decimal(38,10) (nullable = true)
 |-- Credit_History: decimal(38,10) (nullable = true)
 |-- Property_Area: string (nullable = true)
 |-- Loan_Status: integer (nullable = false)
 |-- TotalIncome: decimal(38,6) (nullable = true)



In [29]:
from pyspark.ml.feature import VectorAssembler, OneHotEncoder, StringIndexer
from pyspark.ml import Pipeline

In [30]:
categorical_columns = ['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'Property_Area', 'Credit_History']
numerical_columns = ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term', 'TotalIncome']

# index the string columns
indexers = [StringIndexer(inputCol=col, outputCol="{0}_index".format(col)) for col in categorical_columns]

# encode the indexed values
encoders = [OneHotEncoder(dropLast=False, inputCol=indexer.getOutputCol(), outputCol="{0}_encoded".format(indexer.getOutputCol()))
           for indexer in indexers]

input_columns = [encoder.getOutputCol() for encoder in encoders] + numerical_columns

# vectorize the encoded values
assembler = VectorAssembler(inputCols=input_columns, outputCol="feature")


Öncelikle kategorik sütunların bir listesini ve işlemek istediğiniz sayısal sütunların bir listesini oluşturacağız Kategorik sütunlar StringIndexer kullanılarak dizinlenir. Her kategorik sütun benzersiz dizinlere sahip sayısal bir sütuna dönüştürülür. Her kategorik sütun için ayrı bir StringIndexer oluşturulur ve çıktı sütun adlarına _index eklenir Daha sonra, dizinlenmiş değerler OneHotEncoder kullanılarak kodlanır. Her dizinlenmiş sütun, her kategorinin ayrı bir ikili sütunla temsil edildiği ikili vektör gösterimine dönüştürülür. Yine, her dizinlenmiş sütun için ayrı bir OneHotEncoder oluşturulur ve çıktı sütun adlarına _encoded eklenir Kodlanmış değerler ve sayısal sütunlar, sonraki adımlar için girdi görevi görecek tek bir listede birleştirilir. Son olarak, birleştirilmiş değerler VectorAssembler kullanılarak vektörleştirilir. VectorAssembler, giriş sütunlarının listesini alır ve bunları "features" adlı tek bir özellik vektörü sütununda birleştirir



In [31]:
# verileri dönüştürmek için pipeline oluşturalım
pipeline = Pipeline(stages = indexers + encoders + [assembler])

Pipeline nesnesi, dizinleyicileri, kodlayıcıları ve derleyici adımlarını içeren belirtilen aşamalarla oluşturulur. Aşamalar, + operatörü kullanılarak boru hattına eklenir ve [assembler], tek bir öğe olarak iletildiğinden emin olmak için bir listeye sarılır Pipeline nesnesi, her aşamanın bir dönüşüm veya modelleme adımını temsil ettiği tek bir iş akışında birden fazla aşamayı bir araya getirmenize olanak tanır. Bu, özellik mühendisliği sürecini kolaylaştırmanızı ve bunu farklı veri kümelerine tutarlı bir şekilde uygulamanızı sağlar

In [32]:
data_model = pipeline.fit(df)

tanımlanan Pipeline nesnesini (pipeline) DataFrame df'ye uydurur. fit() yöntemi, pipeline'ın her aşamasını, ilk aşamadan başlayarak ve sonraki aşamalara devam ederek, verilere sırayla uygular

In [33]:
transformed_df = data_model.transform(df)

boru hattında tanımlanan özellik mühendisliği dönüşümlerini gerçekleştirmek için takılan PipelineModel'i (data_model) DataFrame df'ye uygular. Sonuç, df'den gelen orijinal sütunları ve tasarlanmış özellikleri içeren yeni bir DataFrame transformed_df'dir Bir PipelineModel'in transform() yöntemi, boru hattındaki her dönüşüm aşamasını, ilk aşamadan başlayarak ve her aşamanın dönüştürülmüş çıktısını bir sonraki aşamaya girdi olarak sırayla geçirerek giriş DataFrame'ine uygular

In [34]:
transformed_df.show(1)

25/04/28 15:04:26 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------+------+-------+----------+---------+-------------+---------------+-----------------+--------------+----------------+--------------+-------------+-----------+-----------+------------+-------------+----------------+---------------+-------------------+-------------------+--------------------+--------------------+---------------------+------------------------+-----------------------+---------------------------+---------------------------+----------------------------+--------------------+
| Loan_ID|Gender|Married|Dependents|Education|Self_Employed|ApplicantIncome|CoapplicantIncome|    LoanAmount|Loan_Amount_Term|Credit_History|Property_Area|Loan_Status|TotalIncome|Gender_index|Married_index|Dependents_index|Education_index|Self_Employed_index|Property_Area_index|Credit_History_index|Gender_index_encoded|Married_index_encoded|Dependents_index_encoded|Education_index_encoded|Self_Employed_index_encoded|Property_Area_index_encoded|Credit_History_index_encoded|             feature|
+-

In [35]:
# giriş özelliğini ve çıkış sütunlarını al
transformed_df = transformed_df.select(['feature', 'Loan_Status'])

In [36]:
transformed_df.show()

+--------------------+-----------+
|             feature|Loan_Status|
+--------------------+-----------+
|(23,[0,3,4,8,10,1...|          1|
|(23,[0,2,5,8,10,1...|          0|
|(23,[0,2,4,8,11,1...|          1|
|(23,[0,2,4,9,10,1...|          1|
|(23,[0,3,4,8,10,1...|          1|
|(23,[0,2,6,8,11,1...|          1|
|(23,[0,2,4,9,10,1...|          1|
|(23,[0,2,7,8,10,1...|          0|
|(23,[0,2,6,8,10,1...|          1|
|(23,[0,2,5,8,10,1...|          0|
|(23,[0,2,6,8,10,1...|          1|
|(23,[0,2,6,8,10,1...|          1|
|(23,[0,2,6,8,10,1...|          1|
|(23,[0,3,4,8,10,1...|          0|
|(23,[0,2,6,8,10,1...|          1|
|(23,[0,3,4,8,10,1...|          1|
|(23,[0,3,5,9,10,1...|          1|
|(23,[1,3,4,8,10,1...|          0|
|(23,[0,2,4,9,10,1...|          0|
|(23,[0,2,4,8,10,1...|          1|
+--------------------+-----------+
only showing top 20 rows



In [37]:
# split the data for train and test
train_data, test_data = transformed_df.randomSplit([0.8, 0.2], seed=42)

In [38]:
train_data.show(5)

+--------------------+-----------+
|             feature|Loan_Status|
+--------------------+-----------+
|(23,[0,2,4,8,10,1...|          1|
|(23,[0,2,4,8,10,1...|          1|
|(23,[0,2,4,8,10,1...|          0|
|(23,[0,2,4,8,10,1...|          1|
|(23,[0,2,4,8,10,1...|          1|
+--------------------+-----------+
only showing top 5 rows



# Model Training & Testing


In [39]:
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [40]:
lr = LogisticRegression(featuresCol='feature', labelCol='Loan_Status')
lr_model = lr.fit(train_data)

25/04/28 15:04:29 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
25/04/28 15:04:29 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


LogisticRegression sınıfı ikili sınıflandırma görevleri için kullanılır ve ikili bir sonucun olasılığını modellemek için popüler bir algoritma olan lojistik regresyonu uygular özelliklerCol ve etiketCol argümanları, sırasıyla özellikleri ve hedef değişkeni içeren DataFrame'deki sütunların adlarını belirtir LogisticRegression örneğini oluşturduktan sonra, kod lojistik regresyon modelini eğitim verilerine uydurur. fit() yöntemi LogisticRegression nesnesinde çağrılır ve belirtilen DataFrame'deki özellikleri ve hedef değişkeni kullanarak modeli eğitir Sonuçta ortaya çıkan lr_model, eğitilmiş lojistik regresyon modelini temsil eden LogisticRegressionModel sınıfının bir örneğidir. Bu model, yeni veriler hakkında tahminlerde bulunmak veya performansını değerlendirmek için kullanılabilir

In [41]:
# Test verisi üzerinden tahmin yapalım
predictions = lr_model.transform(test_data)
predictions.show()

+--------------------+-----------+--------------------+--------------------+----------+
|             feature|Loan_Status|       rawPrediction|         probability|prediction|
+--------------------+-----------+--------------------+--------------------+----------+
|(23,[0,2,4,8,10,1...|          1|[-2.0518539331989...|[0.11386518564210...|       1.0|
|(23,[0,2,4,8,10,1...|          1|[-2.1285801579184...|[0.10634985744738...|       1.0|
|(23,[0,2,4,8,10,1...|          1|[-2.0120132831664...|[0.11794736315866...|       1.0|
|(23,[0,2,4,8,10,1...|          1|[-2.0637337778771...|[0.11267199789054...|       1.0|
|(23,[0,2,4,8,10,1...|          1|[-2.0102278779703...|[0.11813323592357...|       1.0|
|(23,[0,2,4,8,10,1...|          1|[-1.6688816492826...|[0.15857334099080...|       1.0|
|(23,[0,2,4,8,10,1...|          1|[-2.1286342270332...|[0.10634471885158...|       1.0|
|(23,[0,2,4,8,10,1...|          1|[-2.0215925597755...|[0.11695441718367...|       1.0|
|(23,[0,2,4,8,10,1...|          

Lojistik regresyon modelinin transform() yöntemi, test verilerini girdi olarak alır ve test_data'dan orijinal sütunların yanı sıra "tahmin" sütunu da dahil olmak üzere ek sütunları içeren yeni bir DataFrame tahminleri üretir

LogisticRegressionModel için bir metrik oluşturacağız

In [42]:
predictions = lr_model.transform(test_data)
auc = BinaryClassificationEvaluator().setLabelCol('Loan_Status')
print('AUC:', str(auc.evaluate(predictions)))

AUC: 0.7499999999999999


In [43]:
rf = RandomForestClassifier(featuresCol='feature', labelCol='Loan_Status')
rf_model = rf.fit(train_data)

In [44]:
predictions = rf_model.transform(test_data)
auc = BinaryClassificationEvaluator().setLabelCol('Loan_Status')
print('AUC:', str(auc.evaluate(predictions)))

AUC: 0.7973744292237442


BinaryClassificationEvaluator, AUC-ROC gibi değerlendirme metriklerini hesaplayarak rastgele orman gibi ikili sınıflandırma modellerinin performansını değerlendirmek için kullanılır evaluate() yöntemi, "tahmin" sütununa ve belirtilen etiket sütununa (bu durumda 'Loan_Status') dayanarak ROC eğrisinin altındaki alanı (AUC-ROC) hesaplar. AUC değeri, rastgele orman modelinin iki sınıf arasında ayrım yapma performansını temsil eder

In [53]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField, StringType, DecimalType, IntegerType
from decimal import Decimal  # Decimal türü için import

# Schema tanımlaması
schema = StructType([
    StructField("Loan_ID", StringType(), True),
    StructField("Gender", StringType(), False),
    StructField("Married", StringType(), False),
    StructField("Dependents", StringType(), False),
    StructField("Education", StringType(), True),
    StructField("Self_Employed", StringType(), False),
    StructField("ApplicantIncome", DecimalType(38, 0), True),
    StructField("CoapplicantIncome", DecimalType(38, 10), True),
    StructField("LoanAmount", DecimalType(38, 10), True),
    StructField("Loan_Amount_Term", DecimalType(38, 10), True),
    StructField("Credit_History", DecimalType(38, 10), True),
    StructField("Property_Area", StringType(), True),
    StructField("Loan_Status", IntegerType(), False),
    StructField("TotalIncome", DecimalType(38, 6), True)
])

# Veri (decimal türünde değerler kullanıyoruz)
new_data = [
    Row(
        Loan_ID="LP001002", 
        Gender="Male", 
        Married="No", 
        Dependents="0", 
        Education="Graduate", 
        Self_Employed="No", 
        ApplicantIncome=Decimal(5849), 
        CoapplicantIncome=Decimal(0.0), 
        LoanAmount=Decimal(146.4121621622), 
        Loan_Amount_Term=Decimal(360.0), 
        Credit_History=Decimal(1.0), 
        Property_Area="Urban", 
        Loan_Status=1, 
        TotalIncome=Decimal(5849.0)
    )
]

# DataFrame oluşturuluyor
new_df = spark.createDataFrame(new_data, schema)




In [54]:
new_df.show()

+--------+------+-------+----------+---------+-------------+---------------+-----------------+--------------+----------------+--------------+-------------+-----------+-----------+
| Loan_ID|Gender|Married|Dependents|Education|Self_Employed|ApplicantIncome|CoapplicantIncome|    LoanAmount|Loan_Amount_Term|Credit_History|Property_Area|Loan_Status|TotalIncome|
+--------+------+-------+----------+---------+-------------+---------------+-----------------+--------------+----------------+--------------+-------------+-----------+-----------+
|LP001002|  Male|     No|         0| Graduate|           No|           5849|     0.0000000000|146.4121621622|  360.0000000000|  1.0000000000|        Urban|          1|5849.000000|
+--------+------+-------+----------+---------+-------------+---------------+-----------------+--------------+----------------+--------------+-------------+-----------+-----------+



In [55]:
new_transformed_df = data_model.transform(new_df)


In [56]:
new_transformed_df.show()

+--------+------+-------+----------+---------+-------------+---------------+-----------------+--------------+----------------+--------------+-------------+-----------+-----------+------------+-------------+----------------+---------------+-------------------+-------------------+--------------------+--------------------+---------------------+------------------------+-----------------------+---------------------------+---------------------------+----------------------------+--------------------+
| Loan_ID|Gender|Married|Dependents|Education|Self_Employed|ApplicantIncome|CoapplicantIncome|    LoanAmount|Loan_Amount_Term|Credit_History|Property_Area|Loan_Status|TotalIncome|Gender_index|Married_index|Dependents_index|Education_index|Self_Employed_index|Property_Area_index|Credit_History_index|Gender_index_encoded|Married_index_encoded|Dependents_index_encoded|Education_index_encoded|Self_Employed_index_encoded|Property_Area_index_encoded|Credit_History_index_encoded|             feature|
+-

In [62]:
predictions2 = lr_model.transform(new_transformed_df)
predictions2.select("prediction").show()


+----------+
|prediction|
+----------+
|       1.0|
+----------+

